# Contig quality check

In [2]:
import pandas as pd

sampled_df = pd.read_csv("sampled_metadata.csv")

In [7]:
import sys
!{sys.executable} -m pip install biopython


Defaulting to user installation because normal site-packages is not writeable


In [9]:
pip show biopython

Name: biopython
Version: 1.85
Summary: Freely available tools for computational molecular biology.
Home-page: https://biopython.org/
Author: The Biopython Contributors
Author-email: biopython@biopython.org
License: 
Location: /home/baesuj/.local/lib/python3.11/site-packages
Requires: numpy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


Total number of contigs, largest contig, total assembly size, GC content, N50, L50

In [13]:
import os
import gzip
import pandas as pd
from Bio import SeqIO
import time

In [7]:
# 🗂️ 1. Define the function to extract stats from a FASTA (.fna.gz) file
def get_assembly_stats(fasta_gz_path):
    contig_lengths = []
    total_length = 0
    gc_count = 0

    with gzip.open(fasta_gz_path, "rt") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            seq = record.seq.upper()
            length = len(seq)
            contig_lengths.append(length)
            total_length += length
            gc_count += seq.count("G") + seq.count("C")

    if not contig_lengths:
        return 0, 0, 0, 0, 0, 0

    contig_count = len(contig_lengths)
    largest_contig = max(contig_lengths)
    gc_content = (gc_count / total_length) * 100 if total_length else 0

    contig_lengths.sort(reverse=True)
    half_total = total_length / 2
    running_sum = 0
    n50 = 0
    l50 = 0

    for i, length in enumerate(contig_lengths):
        running_sum += length
        if running_sum >= half_total:
            n50 = length
            l50 = i + 1
            break

    return contig_count, largest_contig, total_length, round(gc_content, 2), n50, l50

In [5]:
# 📍 2. Path to the directory with your .fna.gz files
assembly_dir = "/pool001/robcli/hmp2"  # <- CHANGE THIS

# 📋 3. Load your metadata.csv (must have "External ID" column)
sampled_df = pd.read_csv("sampled_metadata.csv")

# 🧼 4. Make sure "External ID" column is string and has no file extensions
sampled_df["External ID"] = sampled_df["External ID"].astype(str)

In [14]:
# ⏱️ Start timing
start_time = time.time()

# 📦 5. Iterate only over External IDs in sampled_df
results = []

for sample_id in sampled_df["External ID"]:
    filename = f"{sample_id}_contigs.fna.gz"
    fasta_path = os.path.join(assembly_dir, filename)

    if not os.path.isfile(fasta_path):
        print(f"❌ {sample_id}: File not found")
        continue

    try:
        contig_count, largest_contig, total_length, gc_content, n50, l50 = get_assembly_stats(fasta_path)
        results.append({
            "External ID": sample_id,
            "contig_count": contig_count,
            "largest_contig": largest_contig,
            "total_length": total_length,
            "gc_content": gc_content,
            "N50": n50,
            "L50": l50
        })
        print(f"✅ {sample_id}: Stats computed successfully")
    except Exception as e:
        print(f"⚠️ {sample_id}: Error during processing - {e}")
        
end_time = time.time()
elapsed_time = end_time - start_time
print(f"⏱️ Total time taken: {elapsed_time:.2f} seconds")

✅ CSM5MCXD: Stats computed successfully
✅ CSM5MCYS: Stats computed successfully
✅ CSM79HG5: Stats computed successfully
✅ CSM67U9J: Stats computed successfully
✅ CSM67UGC: Stats computed successfully
✅ CSM67UBR: Stats computed successfully
✅ CSM5MCVL: Stats computed successfully
✅ CSM67UBH: Stats computed successfully
✅ CSM79HJW: Stats computed successfully
✅ CSM67UBF: Stats computed successfully
✅ CSM5FZ4M: Stats computed successfully
✅ CSM5MCWQ: Stats computed successfully
✅ CSM67UBX: Stats computed successfully
✅ CSM79HLM: Stats computed successfully
✅ CSM67UBZ: Stats computed successfully
✅ CSM67UDJ: Stats computed successfully
✅ CSM79HLA: Stats computed successfully
✅ CSM67UDR: Stats computed successfully
✅ CSM5MCXN: Stats computed successfully
✅ CSM79HLE: Stats computed successfully
✅ CSM5MCY4: Stats computed successfully
✅ CSM5MCY8: Stats computed successfully
✅ CSM67UEA: Stats computed successfully
✅ CSM67UE7: Stats computed successfully
✅ CSM67UE3: Stats computed successfully


NameError: name 'elapsed_time' is not defined

In [17]:
# 📊 6. Convert results to DataFrame
contig_stats_df = pd.DataFrame(results)

# 🔗 7. Merge with metadata
sampled_df_with_stats = sampled_df.merge(contig_stats_df, on="External ID", how="left")

# 💾 8. Save the updated metadata with new columns
sampled_df_with_stats.to_csv("sampled_metadata_assembly_stats.csv", index=False)
contig_stats_df.to_csv("sampled_assembly_stats.csv", index=False)


## Visualize